In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
""" Klasör oluşturma fonksiyonu """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
import random
import os
import glob
import tensorflow as tf
import sys
sys.path.append('/content/drive/MyDrive')
from unet import build_unet

# -----------------------------
# Parametreler
# -----------------------------
NUM_CLASSES = 4
BATCH_SIZE = 8
INPUT_SHAPE = (128, 128, 3)
NUM_EPOCHS = 5
SAMPLE_SIZE = 1000  # Sadece 100 görüntü rastgele alınacak

# -----------------------------
# Verileri Al
# -----------------------------
image_path = "/content/drive/MyDrive/patches_unzipped/patches/images"
mask_path = "/content/drive/MyDrive/patches_unzipped/patches/masks"

image_paths = sorted(glob.glob(os.path.join(image_path, "*.png")))
mask_paths = sorted(glob.glob(os.path.join(mask_path, "*.png")))

# Rastgele 100 örnek seç
combined = list(zip(image_paths, mask_paths))
random.shuffle(combined)
sampled = combined[:SAMPLE_SIZE]
image_paths, mask_paths = zip(*sampled)

# -----------------------------
# Ön işlem fonksiyonu
# -----------------------------
def preprocess(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, INPUT_SHAPE[:2])
    image = tf.cast(image, tf.float32) / 255.0

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, INPUT_SHAPE[:2], method='nearest')
    mask = tf.one_hot(tf.squeeze(mask, axis=-1), NUM_CLASSES)

    return image, mask

# -----------------------------
# Dataset oluştur
# -----------------------------
def create_dataset(image_paths, mask_paths, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((list(image_paths), list(mask_paths)))
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(100)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = create_dataset(image_paths, mask_paths, BATCH_SIZE)
valid_dataset = train_dataset.take(100)  # küçük bir doğrulama seti

# -----------------------------
# Model eğitimi
# -----------------------------
model = build_unet(INPUT_SHAPE, NUM_CLASSES)
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=["accuracy"]
)

model_path = "/content/drive/MyDrive/model.h5"  # veya istediğin bir isim ve konum
csv_path = "/content/drive/MyDrive/log.csv"     # CSV log dosyası için yol


callbacks = [
    tf.keras.callbacks.ModelCheckpoint(model_path, save_best_only=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1),
    tf.keras.callbacks.CSVLogger(csv_path, append=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=NUM_EPOCHS,
    callbacks=callbacks
)


Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.2391 - loss: 0.1818 
Epoch 1: val_loss improved from inf to 0.36743, saving model to /content/drive/MyDrive/model.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 2360s 18s/step - accuracy: 0.2395 - loss: 0.1812 - val_accuracy: 0.0112 - val_loss: 0.3674 - learning_rate: 1.0000e-04
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.4148 - loss: 0.0461 
Epoch 2: val_loss improved from 0.36743 to 0.27520, saving model to /content/drive/MyDrive/model.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 2259s 18s/step - accuracy: 0.4146 - loss: 0.0461 - val_accuracy: 0.2640 - val_loss: 0.2752 - learning_rate: 1.0000e-04
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.3590 - loss: 0.0374 
Epoch 3: val_loss improved from 0.27520 to 0.18097, saving model to /content/drive/MyDrive/model.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 2293s 18s/step - accuracy: 0.3591 - loss: 0.0375 - val_accuracy: 0.4167 - val_loss: 0.1810 - learning_rate: 1.0000e-04
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.3680 - loss: 0.0323 
Epoch 4: val_loss improved from 0.18097 to 0.08815, saving model to /content/drive/MyDrive/model.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 2223s 18s/step - accuracy: 0.3682 - loss: 0.0323 - val_accuracy: 0.2679 - val_loss: 0.0882 - learning_rate: 1.0000e-04
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.4360 - loss: 0.0383 
Epoch 5: val_loss improved from 0.08815 to 0.04173, saving model to /content/drive/MyDrive/model.h5


125/125 ━━━━━━━━━━━━━━━━━━━━ 2327s 19s/step - accuracy: 0.4360 - loss: 0.0383 - val_accuracy: 0.2951 - val_loss: 0.0417 - learning_rate: 1.0000e-04
